# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [99]:
import pandas as pd

def cargar_datos():
    columnas = [('fecha', 10), ('producto', 30), ('precio', 10), ('cantidad', 5)]
    
    col_specs = [(sum([c[1] for c in columnas[:i]]), sum([c[1] for c in columnas[:i+1]])) for i in range(len(columnas))]
    
    df = pd.read_fwf('datos.dat', colspecs=col_specs, header=None, names=[c[0] for c in columnas])
    
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['precio'] = df['precio'].astype(float)
    df['cantidad'] = df['cantidad'].astype(int)
    
    return df

datos = cargar_datos()

print(datos.head())

       fecha    producto  precio  cantidad
0 2024-08-27     Mirinda  1510.0        14
1 2024-08-27     Mirinda  1560.0        12
2 2024-08-28     Torasso   940.0         8
3 2024-08-29  Pepsi Cola  1210.0        10
4 2024-08-30     Mirinda  1520.0         1


### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [100]:

def calcular_totales(datos):

    datos['importe'] = datos['precio'] * datos['cantidad']
    importe_total = datos['importe'].sum()
    
    cantidad_total = datos['cantidad'].sum()
    
    return importe_total, cantidad_total

importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [101]:

def unidades_vendidas(datos):
    ventas_por_producto = datos.groupby('producto')['cantidad'].sum().sort_values(ascending=False)
    
    resultado = ventas_por_producto.reset_index()
    resultado.columns = ['Producto', 'Unidades Vendidas']
        
    return resultado

# Asumiendo que 'datos' es el DataFrame que obtuvimos de cargar_datos()
unidades_vendidas(datos)

,Producto,Unidades Vendidas
0,Pepsi Cola,89
1,Mirinda,85
2,Sprite,72
3,Coca Cola,57
4,Torasso,32


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [102]:
def precio_promedio(datos):
    promedio_por_producto = datos.groupby('producto')['precio'].mean().sort_values(ascending=False)
    
    resultado = promedio_por_producto.reset_index()
    resultado.columns = ['Producto', 'Precio Promedio']
    
    resultado['Precio Promedio'] = resultado['Precio Promedio'].round(2)
        
    return resultado

precio_promedio(datos)

,Producto,Precio Promedio
0,Mirinda,1545.83
1,Pepsi Cola,1245.00
2,Coca Cola,1072.50
3,Torasso,920.00
4,Sprite,841.43


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [103]:
def ranking_productos(datos, top=3):
    ventas_por_producto = datos.groupby('producto')['cantidad'].sum().sort_values(ascending=False)
    
    top_productos = ventas_por_producto.head(top)
    
    resultado = top_productos.reset_index()
    resultado.columns = ['Producto', 'Unidades Vendidas']
    
    resultado.insert(0, 'Ranking', range(1, len(resultado) + 1))
        
    return resultado

ranking_productos(datos)

,Ranking,Producto,Unidades Vendidas
0,1,Pepsi Cola,89
1,2,Mirinda,85
2,3,Sprite,72


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [104]:
def ventas_por_mes(datos):
    datos['fecha'] = pd.to_datetime(datos['fecha'])
    
    datos['mes'] = datos['fecha'].dt.to_period('M')
    
    ventas = datos.groupby(['producto', 'mes'])['cantidad'].sum().unstack(level='mes')
    
    ventas = ventas.fillna(0)
    
    ventas = ventas.sort_index(axis=1)
    
    ventas['Total'] = ventas.sum(axis=1)
    
    ventas = ventas.sort_values('Total', ascending=False)
    
    ventas = ventas.astype(int)
    
    
    return ventas

ventas_por_mes(datos)

mes,2024-08,2024-09,Total
producto,,,
Pepsi Cola,10,79,89
Mirinda,27,58,85
Sprite,0,72,72
Coca Cola,0,57,57
Torasso,8,24,32


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [105]:
def resumen_ventas(datos):
    precio_promedio = datos.groupby('producto')['precio'].mean()
    
    unidades_vendidas = datos.groupby('producto')['cantidad'].sum()
    
    datos['importe'] = datos['precio'] * datos['cantidad']
    importe_total = datos.groupby('producto')['importe'].sum()
    
    resumen = pd.DataFrame({
        'Precio Promedio': precio_promedio,
        'Unidades Vendidas': unidades_vendidas,
        'Importe Total': importe_total
    })
    
    resumen = resumen.sort_index()
    
    resumen['Precio Promedio'] = resumen['Precio Promedio'].round(2)
    resumen['Importe Total'] = resumen['Importe Total'].round(2)
    
    
    return resumen

resumen_ventas(datos)

,Precio Promedio,Unidades Vendidas,Importe Total
producto,,,
Coca Cola,1072.50,57,60780.0
Mirinda,1545.83,85,131080.0
Pepsi Cola,1245.00,89,110510.0
Sprite,841.43,72,61040.0
Torasso,920.00,32,29320.0


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [106]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

def error(y, y_pred):
    return np.sum(np.abs(y - y_pred))

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

def buscar_coeficientes():
    for a in range(-10, 11):
        for b in range(-10, 11):
            for c in range(-10, 11):
                coeficientes = np.array([a, b, c])
                y_pred = f(X, coeficientes)
                if error(Y, y_pred) == 0:
                    return coeficientes
    return None

coeficientes = buscar_coeficientes()

if coeficientes is not None:
    a, b, c = coeficientes
    print(f"Los coeficientes encontrados son: a = {a}, b = {b}, c = {c}")
    
    print("\nVerificación de los puntos:")
    for x, y in zip(X, Y):
        y_calculada = f(x, coeficientes)
        print(f"Para x = {x}: y calculada = {y_calculada}, y real = {y}")
else:
    print("No se encontró una solución con números enteros pequeños.")

print("\nCoeficientes encontrados:", coeficientes)

Los coeficientes encontrados son: a = -2, b = 10, c = 0

Verificación de los puntos:
Para x = 0: y calculada = 0, y real = 0
Para x = 1: y calculada = 8, y real = 8
Para x = 2: y calculada = 12, y real = 12
Para x = 3: y calculada = 12, y real = 12
Para x = 5: y calculada = 0, y real = 0

Coeficientes encontrados: [-2 10  0]


### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [107]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscar_coeficientes():
    coeficientes = randint(-10, 11, 3)
    
    y_pred = f(X, coeficientes)
    error_actual = error(Y, y_pred)
    
    aprendizaje = 0.001
    
    while error_actual > 1:
        nuevos_coeficientes = coeficientes + np.random.uniform(-1, 1, 3) * aprendizaje
        
        y_pred_nuevo = f(X, nuevos_coeficientes)
        nuevo_error = error(Y, y_pred_nuevo)
        
        if nuevo_error < error_actual:
            coeficientes = nuevos_coeficientes
            error_actual = nuevo_error
    
    return coeficientes

coeficientes = buscar_coeficientes()

print("Coeficientes encontrados:", coeficientes)
print("Error final:", error(Y, f(X, coeficientes)))

print("\nVerificación de los puntos:")
for x, y in zip(X, Y):
    y_calculada = f(x, coeficientes)
    print(f"Para x = {x}: y calculada = {y_calculada:.2f}, y real = {y}")

Coeficientes encontrados: [-1.77998719  8.96197415  0.62605864]
Error final: 0.9998525496879495

Verificación de los puntos:
Para x = 0: y calculada = 0.63, y real = 0
Para x = 1: y calculada = 7.81, y real = 8
Para x = 2: y calculada = 11.43, y real = 12
Para x = 3: y calculada = 11.49, y real = 11
Para x = 5: y calculada = 0.94, y real = 1
